In [2]:
ORM_TYPE="postgres"
ORM_HOST="163.152.71.164"
ORM_PORT="51432"
ORM_USERNAME="user"
ORM_PASSWORD="xhahr123!"
ORM_DATABASE="postgres"
ORM_TABLENAME="nimbus"

In [3]:
from urllib.parse import urlparse
from langchain.vectorstores.pgvector import PGVector
import psycopg2
from tomok import RuleUnitController
import json
import re
import os

def connect_simple():
    url = PGVector.connection_string_from_db_params(
        driver='psycopg2',
        host=ORM_HOST,
        port=ORM_PORT,
        database=ORM_DATABASE,
        user=ORM_USERNAME,
        password=ORM_PASSWORD
    )
    result = urlparse(url)
    conn = psycopg2.connect(
        database=result.path[1:],
        user=result.username,
        password=result.password,
        host=result.hostname,
        port=result.port
    )
    return conn

In [4]:
def find_py_files_and_content(root_dir):
    files_and_contents = []
    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith('.py'):
                # 파일 경로에서 root 디렉토리를 제거
                relative_path = os.path.relpath(os.path.join(root, file), root_dir)
                # 파일 이름에서 .py 확장자 제거
                base_name = os.path.splitext(file)[0]
                # 경로를 디렉토리와 파일 이름으로 분할
                directories = relative_path.split(os.sep)[:-1]  # 파일 이름 제외
                directories.append(base_name)

                # 파일 내용 읽기
                file_path = os.path.join(root, file)
                with open(file_path, 'r', encoding='utf-8') as f:
                    content = f.read()

                # 경로와 내용을 튜플로 묶어 리스트에 추가
                files_and_contents.append((directories, content))

    return files_and_contents
# 사용 예
root_directory = '../ruleunits'  # 여기에 원하는 디렉토리 경로를 넣으세요.
py_paths_contents  = find_py_files_and_content(root_directory)[1:]


def build_classname(dir_list):
    return dir_list[0].upper()+dir_list[1]+'_'+dir_list[2]

# # 출력 예
# for path, contents in py_paths_contents[:3]:
#     if len(path) > 2:
#         print(path)
#         print(build_classname(path))
#         print(contents)

In [5]:
def get_values(ru):
    fc = ru.flowchart
    metadata = {}
    metadata['priority'] = ru.priority
    metadata['version'] = ru.version
    metadata['author'] = ru.author
    metadata['ref_code'] = ru.ref_code
    metadata['ref_date'] = ru.ref_date
    metadata['doc_date'] = ru.doc_date
    metadata['title'] = ru.title
    metadata['description'] = ru.description
    content = ru.content
    
    return fc, metadata, content

In [6]:
controller = RuleUnitController(path='../ruleunits')

values = []
contents = []

for path, content in py_paths_contents:
    if len(path) > 2:
        try:
            values.append(get_values(
                controller[path[0]][path[1]][path[2]][build_classname(path)]
            ))
            contents.append(content)
        except:
            continue

In [7]:
def get_function_info(content: str):
    lines = content.split('\n')
    
    f_string = ""
    flag = False
    
    for line in lines[:-3]:
        if flag:
            f_string += "\n" + line
    
        if "@rule_method" in line:
            flag = True
    
    f_string = f_string.strip()
    # print(f_string)

    lines = f_string.split('\n')
    function_def = lines[0].split(' ')[1].split('(')
    function_name = function_def[0]
    function_vars = function_def[1][:-1]
    
    return f_string, function_name, function_vars

get_function_info(contents[0])

('def Strength_of_longitudinally_reinforcedstruts(fOFns,fIfprims,fIAprims,fIAc,fIfce) -> float:\n        """종방향으로 보강된 스트럿의 강도\n\n        Args:\n            fOFns (float): 종방향으로 보강된 스트럿의 강도\n            fIfprims (float): 압축철근의 응력\n            fIAprims (float): 철근스트럿의 단면적\n            fIAc (float): 스트럿의 유효단면적\n            fIfce (float): 스트럿 또는 절점영역의 콘크리트 유효압축강도\n\n        Returns:\n            float: 콘크리트 스트럿-타이모델 기준  4.2.4 철근 효과 (2)의 값\n        """\n        fOFns=fIfce*fIAc+1.13*fIAprims*fIfprims\n        return fOFns',
 'Strength_of_longitudinally_reinforcedstruts',
 'fOFns,fIfprims,fIAprims,fIAc,fIfce')

In [8]:
values[0][1]['ref_code'][:12]

'KDS 14 20 24'

In [9]:
values[30][1]['description'].split("\n")[1].strip()

'강교 설계기준(한계상태설계법)'

In [14]:
for i in range(len(values)):
    try:
        conn = connect_simple()
        cur = conn.cursor()
        
        code = values[i][1]['ref_code'][:12]
        cur.execute("SELECT * FROM standard_book WHERE code = %s", (code,))
        exists = cur.fetchone()

        if not exists:
            insert_query = """
            INSERT INTO standard_book (std_book_id, code, title, category) 
            VALUES (uuid_generate_v4(), %s, %s, %s);
            """
            
            title = values[i][1]['description'].split("\n")[1].strip()
            print(code, title)
            cur.execute(insert_query, (code, title, None))
            conn.commit()
        else:
            print("already exists, ", i)
    except:
        print("Error", i)
    finally:
        cur.close()
        conn.close()

KDS 14 20 24 콘크리트구조 스트럿-타이모델 기준
already exists,  1
already exists,  2
already exists,  3
already exists,  4
already exists,  5
already exists,  6
already exists,  7
already exists,  8
already exists,  9
already exists,  10
already exists,  11
already exists,  12
already exists,  13
already exists,  14
already exists,  15
already exists,  16
already exists,  17
already exists,  18
already exists,  19
already exists,  20
already exists,  21
already exists,  22
already exists,  23
already exists,  24
already exists,  25
already exists,  26
already exists,  27
already exists,  28
already exists,  29
already exists,  30
already exists,  31
already exists,  32
already exists,  33
already exists,  34
already exists,  35
already exists,  36
already exists,  37
already exists,  38
already exists,  39
already exists,  40
already exists,  41
already exists,  42
already exists,  43
already exists,  44
already exists,  45
already exists,  46
already exists,  47
already exists,  48
already exists,  